## House Regression Analysis using Simple Linear Regression

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train

In [ ]:
test=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test

In [ ]:
train['SalePrice'].describe()

In [ ]:
print("Skewness of the Saleprice target variable is: %f" %train['SalePrice'].skew())
print("Kurtosis of the Saleprice target variable is: %f" %train['SalePrice'].kurt())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use(style="ggplot")
plt.rcParams['figure.figsize']=(10,6)

In [ ]:
plt.hist(train['SalePrice'],color='blue')
plt.show()
sns.distplot(train['SalePrice'],color='blue')

In [ ]:
target=np.log(train['SalePrice'])
print("Skewness of the Saleprice target variable is: %f" %target.skew())
plt.hist(target,color="blue")
plt.show()

In [ ]:
numeric_features=train.select_dtypes(include=[np.number])
print(numeric_features.dtypes)
print(len(numeric_features.dtypes))

In [ ]:
corr=numeric_features.corr()
print(corr['SalePrice'].sort_values(ascending=False)[0:38],'\n')
print(corr['SalePrice'].sort_values(ascending=False)[-5:0],'\n')

In [ ]:
train['OverallQual'].dtype

In [ ]:
train['OverallQual'].unique()

In [ ]:
quality_pivot=train.pivot_table(index='OverallQual',values='SalePrice',aggfunc=np.median)
quality_pivot

In [ ]:
quality_pivot.plot(kind='bar',color='blue')
plt.xlabel('Index:Overall Quality')
plt.ylabel('Median')
plt.show()

In [ ]:
plt.scatter(train['GrLivArea'],target)
plt.xlabel('Ground Living Area(in sqaure feet)')
plt.ylabel('Log Transformed SalePrice')
plt.show()
plt.scatter(train['GarageArea'],target)
plt.xlabel('Garage Area')
plt.ylabel('Log Transformed SalePrice')
plt.show()

In [ ]:
train=train[train['GarageArea']<1200]
plt.scatter(train['GarageArea'],np.log(train['SalePrice']))
plt.xlim(-200,1600)
plt.xlabel('Garage area')
plt.ylabel('Sale price')
plt.show()
print(train['GarageArea'].shape)
print(np.log(train['SalePrice']).shape)

In [ ]:
nulls=pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[0:30])
nulls.columns=['Null count']
nulls.index.name='Features'
nulls

In [ ]:
objects=train.select_dtypes(exclude=[np.number])
objects.columns

In [ ]:
print(train['Street'].value_counts())
train['encode_street']=pd.get_dummies(train['Street'],drop_first=True)
test['encode_street']=pd.get_dummies(test['Street'],drop_first=True)
print(train['encode_street'].value_counts())

In [ ]:
def encode(x):
    return 1 if x=='Partial' else 0
train['encode_salecondition']=train['SaleCondition'].apply(encode)
test['encode_salecondition']=test['SaleCondition'].apply(encode)

In [ ]:
sale_cond_pivot=train.pivot_table(index='encode_salecondition',values='SalePrice',aggfunc=np.median)
sale_cond_pivot.plot(kind='bar',color='blue')

In [ ]:
data=train.select_dtypes(include=[np.number]).interpolate().dropna()

In [ ]:
sum(data.isnull().sum()!=0)

In [ ]:
y=np.log(data['SalePrice'])
X=data.drop(['Id','SalePrice'],axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
print(lr.score(X_test,y_test))

In [ ]:
predictions=lr.predict(X_test)
predictions

In [ ]:
test_data=test.select_dtypes(include=[np.number]).drop(['Id'],axis=1).interpolate()

In [ ]:
pred=lr.predict(test_data)
pred

In [ ]:
final_predictions=np.exp(pred)
final_predictions

In [ ]:
output=pd.DataFrame({'Id':test['Id'],'SalePrice':final_predictions})
output.to_csv("submission.csv",index=False)

In [ ]:
output.head(10)